# Assignment 4 - Using NLP to play the stock market

In this assignment, we'll use everything we've learned to analyze corporate news and pick stocks. Be aware that in this assignment, we're trying to beat the benchmark of random chance (aka better than 50%).

This assignment will involve building three models:

**1. An RNN based on word inputs**

**2. A CNN based on character inputs**

**3. A neural net architecture that merges the previous two models**

You will apply these models to predicting whether a stock return will be positive or negative in the same day of a news publication.

## Your X - Reuters news data

Reuters is a news outlet that reports on corporations, among many other things. Stored in the `news_reuters.csv` file is news data listed in columns. The corresponding columns are the `ticker`, `name of company`, `date of publication`, `headline`, `first sentence`, and `news category`.

In this assignment it is up to you to decide how to clean this dataset. For instance, many of the first sentences contain a location name showing where the reporting is done. This is largely irrevant information and will probably just make your data noisier. You can also choose to subset on a certain news category, which might enhance your model performance and also limit the size of your data.

## Your Y - Stock information from Yahoo! Finance

Trading data from Yahoo! Finance was collected and then normalized using the [S&P 500](https://en.wikipedia.org/wiki/S%26P_500_Index). This is stored in the `stockReturns.json` file. 

In our dataset, the ticker for the S&P is `^GSPC`. Each ticker is compared the S&P and then judged on whether it is outperforming (positive value) or under-performing (negative value) the S&P. Each value is reported on a daily interval from 2004 to now.

Below is a diagram of the data in the json file. Note there are three types of data: short: 1 day return, mid: 7 day return, long 28 day return.

```
          term (short/mid/long)
         /         |         \
   ticker A   ticker B   ticker C
      /   \      /   \      /   \
  date1 date2 date1 date2 date1 date2
```

You will need to pick a length of time to focus on (day, week, month). You are welcome to train models on each dataset as well.  

Transform the return data such that the outcome will be binary:

```
label[y < 0] = 0
label[y >= 0] = 1
```

Finally, this data needs needs to be joined on the date and ticker - For each date of news publication, we want to join the corresponding corporation's news on its return information. We make the assumption that the day's return will reflect the sentiment of the news, regardless of timing.


# Your models - RNN, CNN, and RNN+CNN

For your RNN model, it needs to be based on word inputs, embedding the word inputs, encoding them with an RNN layer, and finally a decoding step (such as softmax or some other choice).

Your CNN model will be based on characters. For reference on how to do this, look at the CNN class demonstration in the course repository.

Finally you will combine the architecture for both of these models, either [merging](https://github.com/ShadyF/cnn-rnn-classifier) using the [Functional API](https://keras.io/getting-started/functional-api-guide/) or [stacking](http://www.aclweb.org/anthology/S17-2134). See the links for reference.

For each of these models, you will need to:
1. Create a train and test set, retaining the same test set for every model
2. Show the architecture for each model, printing it in your python notebook
2. Report the peformance according to some metric
3. Compare the performance of all of these models in a table (precision and recall)
4. Look at your labeling and print out the underlying data compared to the labels - for each model print out 2-3 examples of a good classification and a bad classification. Make an assertion why your model does well or poorly on those outputs.
5. For each model, calculate the return from the three most probable positive stock returns. Compare it to the actual return. Print this information in a table.

### Good luck!

## Load and Clean Raw Data

In [217]:
# Utility libraries
import os
import pickle
import numpy as np
import pandas as pd
import re
import calendar

# Prepocessing libraries
from sklearn.model_selection import train_test_split
import gensim

from keras.models import Model
from keras.layers import Input, concatenate, Concatenate, TimeDistributed
from keras.layers import Dense, Bidirectional, Dropout, Flatten 
from keras.layers import Conv1D, Conv2D, MaxPool1D
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

In [72]:
import keras.backend as K

def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [2]:
dataPath = '../data'
reutersFile = 'news_reuters.csv'
stockFile = 'stockReturns.json'

rawX = pd.read_csv(os.path.join(dataPath, reutersFile), header=None, 
                   names=['ticker', 'company', 'pub_date', 'headline', 'first_sent', 'category'])
rawY = pd.read_json(os.path.join(dataPath, stockFile))
# rawY = json.load(os.path.join(dataPath, stockFile))

#### Reformat and Merge Data

In [3]:
def reformat_y_data(data, tickerType='mid'):
    """Convert stock data into binary postive/negative"""
    tmp = data[tickerType].apply(pd.Series)
    tmp = tmp.stack().rename('price', inplace=True).reset_index()
    tmp['y'] = np.where(tmp['price'] >= 0, 1, 0)
    tmp.rename(columns={'level_0': 'ticker', 'level_1': 'pub_date'}, inplace=True)
    return tmp

def clean_and_merge_data(X, Y):
    """Filter X to only those tickers with stock data"""
    y_tickers = set(Y['ticker'])
    X = X.loc[X['ticker'].isin(y_tickers)]
    # Make sure data types are the same for merge    
    Y['pub_date'] = Y['pub_date'].astype(rawX['pub_date'].dtype)
    Y['ticker'] = Y['ticker'].astype(rawX['ticker'].dtype)
    return X.merge(Y, on=['ticker', 'pub_date'], how='left')

In [4]:
cleanY = reformat_y_data(rawY, 'mid')

merged = clean_and_merge_data(rawX, cleanY)

#### Clean up text columns and tokenize data

In [5]:
def clean_text(sent):
    """Clean up text data by:
    
    1. Replacing double spaces into a single space
    2. Replace U.S. to United States so U won't get deleted with next 
       replacement
    3. Remove all capitalized words at the beginning of the 
       sentence, since those are mostly places (aka NEW YORK)
    4. Remove unnecessary punctuation (hyphens and asterisks)
    5. Remove dates
    """
    monthStrings = list(calendar.month_name)[1:] + list(calendar.month_abbr)[1:]
    monthPattern = '|'.join(monthStrings)
    
    sent = re.sub(r' +', ' ', sent)
    sent = re.sub(r'U.S.', 'United States', sent)
    sent = re.sub(r'^(\W?[A-Z\s\d]+\b-?)', '', sent)
    sent = re.sub(r'^ ?\W ', '', sent)
    sent = re.sub(r'({}) \d+'.format(monthPattern), '', sent)
    
    # replace double spaces one more time after previous cleaning 
    sent = re.sub(r' +', ' ', sent)
    return sent 

In [7]:
def tokenize_sent(col):
    """Tokenize string into a sequence of words"""
    return [text_to_word_sequence(text, lower=False) for text in col]

def filt_to_one(x, random_state=10):
    """Filter dataset so that there is only one observation per day.
    
    If there is more than one record, will use the topStory record
    if one exists.  If one doesn't or there are 2 topStory records
    then it will randomly select one of the observations.
    """
    if x.shape[0] > 1:
        if 'topStory' in x['category'].unique():
            x = x.loc[x['category'] == 'topStory']
        if x.shape[0] > 1:
            x = x.sample(n=1, random_state=random_state)
    return x

In [25]:
# Clean up text
merged['headline'] = merged.headline.apply(clean_text)
merged['first_sent'] = merged.first_sent.apply(clean_text)

# Turn sentences into tokens
merged['headline_token'] = tokenize_sent(merged.headline)
merged['first_sent_token'] = tokenize_sent(merged.first_sent)

# Get one record per company/day
finalData = merged.groupby(by=['ticker', 'pub_date']).apply(filt_to_one)

# Combine Headline and First Sentence into one text 
finalData['final_text'] = finalData['headline_token'] + finalData.first_sent_token

# Remove observations with missing stock price
finalData.dropna(inplace=True)

In [30]:
# split into train and test
train, test = train_test_split(finalData, test_size = .2, random_state=10)

#### Create Lexicon and Transform Data to Integers for Modeling

In [33]:
class lexiconTransformer():
    """Create a lexicon and transform sentences and
       to indexes for use in the model."""
    
    def __init__(self, words_min_freq = 1, unknown_word_token = u'<UNK>',
                 savePath='models', saveName='stock_word_lexicon'):
        self.words_min_freq = words_min_freq
        self.words_lexicon = None
        self.unknown_word_token = unknown_word_token
        self.indx_to_words_dict = None
        self.savePath = savePath
        self.saveName = saveName + '.pkl'
    
    def fit(self, sents):
        """Create lexicon based on sentences"""
        self.make_words_lexicon(sents)        
        self.make_lexicon_reverse()
        self.save_lexicon()
                
    def transform(self, sents):
        sents_indxs = self.tokens_to_idxs(sents, self.words_lexicon)
        return sents_indxs

    def fit_transform(self, sents):
        self.fit(sents)
        return self.transform(sents)
        
    def make_words_lexicon(self, sents_token):
        """Wrapper for words lexicon"""
        self.words_lexicon = self.make_lexicon(sents_token, self.words_min_freq,
                                               self.unknown_word_token)

    def make_lexicon(self, token_seqs, min_freq=1, unknown = u'<UNK>'):
        """Create lexicon from input based on a frequency

            Parameters:
            
            token_seqs
            ----------
               A list of a list of input tokens that will be used to create the lexicon
            
            min_freq
            --------
               Number of times the token needs to be in the corpus to be included in the
               lexicon.  Otherwise, will be replaced with the "unknown" entry
            
            unknown
            -------
               The word in the lexicon that should be used for tokens not existing in lexicon.
               This can be a value that already exists in input list.  For instance, in 
               Named Entity Recognition, a value of "other" or "O" may already be a tag 
               and so having "other" and "unknown" are the same thing!
        """
        # Count how often each word appears in the text.
        token_counts = {}
        for seq in token_seqs:
            for token in seq:
                if token in token_counts:
                    token_counts[token] += 1
                else:
                    token_counts[token] = 1

        # Then, assign each word to a numerical index. 
        # Filter words that occur less than min_freq times.
        lexicon = [token for token, count in token_counts.items() if count >= min_freq]
        
        # Have to delete unknown value from token list so not a gap in lexicon values when
        # turning it into a lexicon (aka, if unknown == OTHER and that is the 7th value, 
        # then 7 won't exist in the lexicon which may cause issues)
        if unknown in lexicon:
            lexicon.remove(unknown)

        # Indices start at 1. 0 is reserved for padding, and 1 is reserved for unknown words.
        lexicon = {token:idx + 2 for idx,token in enumerate(lexicon)}
        
        lexicon[unknown] = 1 # Unknown words are those that occur fewer than min_freq times
        lexicon_size = len(lexicon)
        return lexicon
    
    def save_lexicon(self):
        "Save lexicons by pickling them"
        if not os.path.exists(self.savePath):
            os.makedirs(self.savePath)
        with open(os.path.join(self.savePath, self.saveName), 'wb') as f:
            pickle.dump(self.words_lexicon, f)
                        
    def load_lexicon(self):
        with open(os.path.join(self.savePath, self.saveName), 'rb') as f:
            self.words_lexicon = pickle.load(f)
                    
        self.make_lexicon_reverse()
        
    def make_lexicon_reverse(self):
        self.indx_to_words_dict = self.get_lexicon_lookup(self.words_lexicon)
    
    def get_lexicon_lookup(self, lexicon):
        '''Make a dictionary where the string representation of 
           a lexicon item can be retrieved from its numerical index'''
        lexicon_lookup = {idx: lexicon_item for lexicon_item, idx in lexicon.items()}
        return lexicon_lookup
    
    def tokens_to_idxs(self, token_seqs, lexicon):
        """Transform tokens to numeric indexes or <UNK> if doesn't exist"""
        idx_seqs = [[lexicon[token] if token in lexicon else lexicon['<UNK>'] for 
                                 token in token_seq] for token_seq in token_seqs]
        return idx_seqs

In [42]:
lexicon = lexiconTransformer(words_min_freq=2)

lexicon.fit(train['final_text'])

In [44]:
train['finalText_indx'] = lexicon.transform(train['final_text'])
test['finalText_indx'] = lexicon.transform(test['final_text'])

C:\Users\Hamburg\Anaconda3\envs\tensorflow2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Hamburg\Anaconda3\envs\tensorflow2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [55]:
def get_max_seq_len(sents):
    return max([len(idx_seq) for idx_seq in sents])

def pad_idx_seqs(idx_seqs, max_seq_len):
    # Keras provides a convenient padding function; 
    padded_idxs = pad_sequences(sequences=idx_seqs, maxlen=max_seq_len)
    return padded_idxs

In [111]:
# Get length of longest sequence
max_seq_len = get_max_seq_len(train['finalText_indx'])

#Add one to max length for offsetting sequence by 1
train_padded_words = pad_idx_seqs(train['finalText_indx'], 
                                  max_seq_len + 1) 

test_padded_words = pad_idx_seqs(test['finalText_indx'], 
                                  max_seq_len + 1) 

train_y = to_categorical(train['y'])
test_y = to_categorical(test['y'])

## Model 1: RNN

In [203]:
n_out = 2
nb_epoch = 10

In [68]:
def create_embed_matrix(model, lexicon, embed_size):
    "Create a weight matrix for words"
    vocab_size = len(lexicon)
    embedding_matrix = np.zeros((vocab_size, embed_size))
    n = 0
    word_list = list(lexicon)
    for i in range(vocab_size):
        word = word_list[i]
        if word in model.wv.vocab:
            embedding_vector = model.wv[word]
            if embedding_vector is not None:
                embedding_matrix[n] = embedding_vector[:embed_size]
                n += 1

    return embedding_matrix[:n, :]


In [62]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('../data/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [99]:
word_embed_len = 200
word_embed_matrix = create_embed_matrix(w2v, lexicon.words_lexicon, 
                                   word_embed_len)

In [118]:
def create_rnn_model(seq_input_len, embed_matrix, 
                     n_RNN_nodes, n_dense_nodes, 
                     batch_size=None, recurrent_dropout=0.2, 
                     drop_out=.2, n_out=n_out):
    
    word_input = Input(batch_shape=(batch_size, seq_input_len), name='word_input_layer')
        
    word_embeddings = Embedding(input_dim=embed_matrix.shape[0],
                                output_dim=embed_matrix.shape[1],
                                weights=[embed_matrix], 
                                mask_zero=True, 
                                name='word_embedding_layer')(word_input) 

    hidden_layer = Bidirectional(LSTM(units=n_RNN_nodes, return_sequences=False, 
                                      recurrent_dropout=recurrent_dropout,
                                      name='hidden_layer'))(word_embeddings)

    dense_layer = Dense(units=n_dense_nodes, activation='relu', name='dense_layer')(hidden_layer)

    drop_out3 = Dropout(drop_out)(dense_layer)

    output_layer = Dense(units=n_out, activation='softmax',
                         name='output_layer')(drop_out3)

    model = Model(inputs=[word_input], outputs=output_layer)
    model.compile(loss='categorical_crossentropy', optimizer="adam", 
                  metrics=['accuracy', recall, precision])

    return model 

In [119]:
rnn_model = create_rnn_model(seq_input_len=train_padded_words.shape[-1],
                             embed_matrix=embed_matrix, 
                             n_RNN_nodes=200, n_dense_nodes=200)

In [120]:
def train_and_test_model(model, x_train, y_train, x_test, y_test, 
                         modelSaveName, modelSavePath='models',
                         batch_size=64, epochs=3, validation_split=.1):
    """Train model, save weights, and predict data"""
    
    print(model.summary())
    
    filepath = os.path.join(modelSavePath, modelSaveName + '.hdf5')
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1)
    callbacks_list = [checkpoint]
    model.fit(x=[x_train], y=y_train, batch_size=batch_size, 
              epochs=epochs, validation_split=validation_split, 
              callbacks=callbacks_list)
    
    score, acc, rec, prec = model.evaluate(x_test, y_test, batch_size=batch_size)
    return (model, acc, rec, prec)    

In [121]:
rnn_res = train_and_test_model(rnn_model, train_padded_words, train_y, 
                               test_padded_words, test_y, 'rnn_model',
                               epochs=nb_epoch)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_input_layer (InputLayer (None, 102)               0         
_________________________________________________________________
word_embedding_layer (Embedd (None, 102, 200)          2297400   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 400)               641600    
_________________________________________________________________
dense_layer (Dense)          (None, 200)               80200     
_________________________________________________________________
dropout_5 (Dropout)          (None, 200)               0         
_________________________________________________________________
output_layer (Dense)         (None, 2)                 402       
Total params: 3,019,602
Trainable params: 3,019,602
Non-trainable params: 0
_________________________________________________________________


In [129]:
rnn_res

(<keras.engine.training.Model at 0x29be139df28>,
 0.5050182481751825,
 0.5050182481751825,
 0.5050182481751825)

## Model 2: CNN

In [219]:
def vectorize_sentences(data, lexicon, maxlen=200):
    X = []
    for sentences in data:
        x = [lexicon[token] if token in lexicon else lexicon['<UNK>'] for 
                                 token in sentences]
        x2 = np.eye(len(char_indices) + 1)[x]
        X.append(x2)
    return (pad_sequences(X, maxlen=maxlen))

def create_cnn_model(seq_input_len, batch_size=None, 
                     nb_filter=100, filter_kernels = [4] * 4,
                     pool_size=3, n_dense_nodes=100,
                     drop_out=.2, n_out=2):

    inputs = Input(shape=(batch_size, seq_input_len), name='char_input_layer')

    conv1 = Conv1D(nb_filter, kernel_size=filter_kernels[0],
                  padding='valid', activation='relu',
                  input_shape=(batch_size, seq_input_len))(inputs)
    
    maxpool1 = MaxPool1D(pool_size=pool_size)(conv1)

    conv2 = Conv1D(nb_filter, kernel_size=filter_kernels[1],
                          padding='valid', activation='relu')(maxpool1)
    maxpool2 = MaxPool1D(pool_size=pool_size)(conv2)

    conv3 = Conv1D(nb_filter, kernel_size=filter_kernels[2],
                          padding='valid', activation='relu')(maxpool2)

    conv4 = Conv1D(nb_filter, kernel_size=filter_kernels[3],
                          padding='valid', activation='relu')(conv3)

    maxpool3 = MaxPool1D(pool_size=pool_size)(conv4)
    flatten = Flatten()(maxpool3)

    dense_layer = Dense(n_dense_nodes, activation='relu')(flatten)
    dropout = Dropout(drop_out)(dense_layer)

    output_layer = Dense(n_out, activation='softmax', name='output')(dropout)

    model = Model(inputs=inputs, outputs=output_layer)

    model.compile(loss='categorical_crossentropy', optimizer="adam", 
                  metrics=['accuracy', recall, precision])    
    return model 

In [214]:
char_maxlen = 1024 
nb_filter = 256
dense_outputs = 1024
filter_kernels = [7, 7, 3, 3]
pool_size = 3

In [201]:
# Turn all tokens into one string and then all obs 
# into one overall string
trainTokensAsString = train.final_text.apply(lambda x: ' '.join(x))
testTokensAsString = test.final_text.apply(lambda x: ' '.join(x))
oneTxt = ' '.join(trainTokensAsString)

# Get info about characters
chars = set(oneTxt)
vocab_size = len(chars) + 1
print('total chars:', vocab_size)
char_indices = dict((c, i + 2) for i, c in enumerate(chars))
indices_char = dict((i + 2, c) for i, c in enumerate(chars))

char_indices['<UNK>'] = 1
indices_char[1] = '<UNK>'

total chars: 85


In [206]:
trainCharData = vectorize_sentences(trainTokensAsString, char_indices, char_maxlen)
testCharData = vectorize_sentences(testTokensAsString, char_indices, char_maxlen)

In [220]:
cnn_model = create_cnn_model(batch_size=char_maxlen, 
                             seq_input_len=vocab_size,
                             nb_filter=nb_filter, 
                             filter_kernels=filter_kernels,
                             pool_size=pool_size, 
                             n_dense_nodes=dense_outputs,
                             drop_out=.2, 
                             n_out=n_out)

In [225]:
cnn_res = train_and_test_model(cnn_model, trainCharData[:, :, 1:],
                               train_y, 
                               testCharData[:, :, 1:], 
                               test_y, 
                               'rnn_model',
                               epochs=3)
#                                epochs=nb_epoch)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
char_input_layer (InputLayer (None, 1024, 85)          0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 1018, 256)         152576    
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 339, 256)          0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 333, 256)          459008    
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 111, 256)          0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 109, 256)          196864    
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 107, 256)          196864    
__________

## Model 3: RNN+CNN

(0.5050182481751825, 0.5050182481751825, 0.5050182481751825)

1. Create a train and test set, retaining the same test set for every model
2. Show the architecture for each model, printing it in your python notebook
2. Report the peformance according to some metric
3. Compare the performance of all of these models in a table (precision and recall)
4. Look at your labeling and print out the underlying data compared to the labels - for each model print out 2-3 examples of a good classification and a bad classification. Make an assertion why your model does well or poorly on those outputs.
5. For each model, calculate the return from the three most probable positive stock returns. Compare it to the actual return. Print this information in a table.

#### Put results in a table

In [234]:
pd.DataFrame.from_records([cnn_res[1:4], rnn_res[1:4]], 
                          columns=['accuracy', 'recall', 'precision'], 
                         index=['cnn_mod', 'rnn_mod'])

,accuracy,recall,precision
cnn_mod,0.487226,0.487226,0.487226
rnn_mod,0.505018,0.505018,0.505018
